In [55]:
import numpy as np
import arepo
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from tqdm import tqdm
import matplotlib as mpl
from scipy.stats import binned_statistic_2d
from numba import njit

In [1]:
basepath = '/n/holylfs05/LABS/hernquist_lab/Users/abeane/GSEgas/'

In [6]:
G = 43018.7
H0 = 0.1

In [14]:
# GSE2 structural parameters
V200 = 97.5
MD = 0.007747
MB = 0.0
LAMBDA = 0.008
JD = 0.00955

M200 = V200**3 / (10 * G * H0)
R200 = V200 / (10 * H0)

print(M200)

21.545499399098528


In [9]:
(1/10)**(1/3)

0.4641588833612779

In [10]:
print('V200 = ', V200*(1/10)**(1/3))

V200 =  45.255491127724596


In [11]:
# GSE2 structural parameters
V200 = 129
MD = 0.007747
MB = 0.0
LAMBDA = 0.008
JD = 0.00955

M200 = V200**3 / (10 * G * H0)
R200 = V200 / (10 * H0)

print(M200)

49.90129873752577


In [12]:
49.9/21.55

2.315545243619489

In [16]:
sn_Seq = arepo.Snapshot(basepath + 'ics/Seq1iso_fg0.7/lvl4/Seq_ICs.dat')
sn_GSE = arepo.Snapshot(basepath + 'ics/GSE2iso_fg0.7/lvl4/GSE_ICs.dat')

In [11]:
mbar = 7.5E3 * 8 / 1E10
mdm = 3.75E4 * 8 / 1E10

print('Num gas:', sn.MassTable[0] * sn.NumPart_Total[0]/mbar)
print('Num halo:', sn.MassTable[1] * sn.NumPart_Total[1]/mdm)
print('Num disk:', sn.MassTable[2] * sn.NumPart_Total[2]/mbar)

Num gas: 1953.595899354108
Num halo: 71491.68865078875
Num disk: 837.2553854374751


In [14]:
print(sn.MassTable)

[5.99875916e-06 2.99998693e-05 6.00183072e-06 0.00000000e+00
 0.00000000e+00 0.00000000e+00]


## check tidal radius

In [15]:
def rt_eq2(R_Seq=80.):
    M_Seq = 0.0
    M_GSE = 0.0
    
    for i in range(6):
        M_Seq += sn_Seq.MassTable[i] * sn_Seq.NumPart_Total[i]
        M_GSE += sn_GSE.MassTable[i] * sn_GSE.NumPart_Total[i]
    
    rt = R_Seq * (M_Seq / (3*M_GSE))**(1./3.)
    
    return rt

In [22]:
def Menc(Rcut, sn, center=np.array([0., 0., 0.])):
    ans = 0.0
    for i in range(6):
        if sn.NumPart_Total[i] > 0:
            part = getattr(sn, 'part'+str(i))
            r = np.linalg.norm(part.pos-center, axis=1)
            key = np.where(r < Rcut)[0]
            if sn.MassTable[i] > 0.0:
                ans += sn.MassTable[i] * len(key)
            else:
                ans += np.sum(part.mass[key])
    
    return ans
            
def rt_eq6(R_Seq=80.):
    M_GSE = Menc(R_Seq, sn_GSE)
    # print(M_MW)
    
    # print(M_MW)
    # for a hernquist halo (the only material at R_Sgr)
    rt_guess = rt_eq2(R_Seq)

    # print(R_GSE)
    # print(Menc(rt_guess, sn_GSE))
    
    for _ in range(10):
        rt = R_Seq * ((Menc(rt_guess, sn_Seq)/M_GSE))**(1./3.)
        rt_guess = rt
    
    return rt

In [27]:
R = 6
print(rt_eq2(R))
print(rt_eq6(R))

1.9328778909589999
2.7623469143350894


In [28]:
pos = np.random.rand(100, 3)
np.sum(pos, axis=0)

array([49.72957407, 51.1066462 , 51.0583527 ])

In [30]:
sn_Seq = arepo.Snapshot(basepath + 'ics/GSE2_Seq1/lvl4/Seq.hdf5')
sn_GSE = arepo.Snapshot(basepath + 'ics/GSE2_Seq1/lvl4/GSE.hdf5')

In [32]:
np.median(sn_Seq.part0.pos, axis=0)

array([300.023  , 299.99326, 299.99622], dtype=float32)

In [130]:
sn = arepo.Snapshot(basepath + 'ics/GSE2_Seq1/lvl4/GSE_Seq-with-grid.hdf5')

In [82]:
np.median(sn.part2.pos, axis=0)

Quantity([ 99.41780853, 100.01990509, 100.00286865], dtype=float32, unit=[kpc])

In [91]:
sn.part0.vel[sn.part0.rho < 1e-9][:100]

Quantity([[  0.        ,   0.        ,   0.        ],
          [  0.        ,   0.        ,   0.        ],
          [  0.        ,   0.        ,   0.        ],
          [  0.        ,   0.        ,   0.        ],
          [ 34.72151566, -71.59770966,   0.        ],
          [  0.        ,   0.        ,   0.        ],
          [  0.        ,   0.        ,   0.        ],
          [ 34.72151566, -71.59770966,   0.        ],
          [ 34.72151566, -71.59770966,   0.        ],
          [-13.26264477, -77.79891968,   0.        ],
          [ 34.72151566, -71.59770966,   0.        ],
          [-20.09971809, -81.03945923,   0.        ],
          [  0.        ,   0.        ,   0.        ],
          [  0.        ,   0.        ,   0.        ],
          [-14.59528637, -82.23511505,   0.        ],
          [ 11.18181515, -82.34670258,   0.        ],
          [  4.70169592, -81.69868469,   0.        ],
          [ 19.21825981, -79.597435  ,   0.        ],
          [ 18.4821949 , -80

In [67]:
@njit
def my_mult(A, B):
    N = A.shape[0]
    M = B.shape[1]

    out = np.zeros(B.shape)

    for i in range(N):
        for j in range(M):
            out[i][j] = A[i] * B[i][j]

    return out


In [68]:
my_mult(sn_GSE.part0.mass, sn_GSE.part0.pos)

array([[0.00180004, 0.00180004, 0.00180004],
       [0.00179721, 0.00180087, 0.00180009],
       [0.001795  , 0.00179819, 0.00179913],
       ...,
       [0.00180113, 0.00180258, 0.0017992 ],
       [0.00178638, 0.00179422, 0.00179909],
       [0.00180374, 0.00179677, 0.00179992]])

In [74]:
np.mean(sn.part2.vel, axis=0)

array([-0.64278907,  0.41684803, -0.14441285], dtype=float32)

In [83]:
sn.part0.u

Quantity([  0.     ,   0.     ,   0.     , ..., 210.48248291,
          210.48248291, 210.48248291], dtype=float32, unit=[km^2 s^-2])

In [100]:
key = np.where(sn.part0.id==815135)[0][0]
sn.part0.pos[key]

Quantity([100.0243454,  56.08411026,  62.21882248], dtype=float32, unit=[kpc])

In [123]:
def get_profile(sn, ptype):
    assert sn.NumPart_Total[ptype] > 0
    
    part = getattr(sn, 'part'+str(ptype))
    center = np.array([sn.BoxSize, sn.BoxSize, sn.BoxSize])/2.
    
    r = np.linalg.norm(part.pos.value - center, axis=1)
    
    rmin = 0.
    dr = 0.5
    
    rlist = []
    rholist = []
    
    while rmin < 50:
        key = np.logical_and(r > rmin, r < rmin + dr)
        mass = sn.MassTable[ptype] * len(np.where(key)[0])
        vol = (4*np.pi/3.) * ((rmin+dr)**3 - rmin**3)
        
        rholist.append(mass/vol)
        rlist.append(np.mean(r[key]))
        
        rmin += dr
    
    return np.array(rlist), np.array(rholist)
    

In [128]:
r, rho = get_profile(sn, 1)

In [129]:
sn.MassTable

Quantity([0., 0., 0., 0., 0., 0.], unit=[1e+10 Msol])

In [133]:
sn_pregrid = arepo.Snapshot(basepath + 'ics/GSE2_Seq1/lvl4/GSE_Seq.hdf5')

In [134]:
sn_pregrid.MassTable

array([0.00000000e+00, 3.00000279e-05, 5.99992747e-06, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00])